# Análise comparativa de modelos

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Leitura da base de dados
df = pd.read_csv("../data/raw/doencas_cardiacas.csv")

In [3]:
# Colunas desnecessárias
colunas_a_dropar = ["age", "resting bp s", "cholesterol", "max heart rate", "oldpeak"]

# Dropando colunas desnecessárias
df = df.drop(columns = colunas_a_dropar)

# Colunas categóricas e suas categorias esperadas
colunas_categoricas = [
    "sex",
    "chest pain type",
    "fasting blood sugar",
    "resting ecg",
    "exercise angina",
    "ST slope",
    "target"
]

# Aplicar One Hot Encoding usando pandas.get_dummies
df_encoded = pd.get_dummies(df, columns=colunas_categoricas, drop_first=False)

# Salvar o novo dataframe codificado
df_encoded.to_csv("../data/processed/doencas_cardiacas_one_hot.csv", index=False)

# Exibir a nova matriz
print(df_encoded)

      sex_0  sex_1  chest pain type_1  chest pain type_2  chest pain type_3  \
0         0      1                  0                  1                  0   
1         1      0                  0                  0                  1   
2         0      1                  0                  1                  0   
3         1      0                  0                  0                  0   
4         0      1                  0                  0                  1   
...     ...    ...                ...                ...                ...   
1185      0      1                  1                  0                  0   
1186      0      1                  0                  0                  0   
1187      0      1                  0                  0                  0   
1188      1      0                  0                  1                  0   
1189      0      1                  0                  0                  1   

      chest pain type_4  fasting blood sugar_0  fas

In [4]:
print(df_encoded.columns)

Index(['sex_0', 'sex_1', 'chest pain type_1', 'chest pain type_2',
       'chest pain type_3', 'chest pain type_4', 'fasting blood sugar_0',
       'fasting blood sugar_1', 'resting ecg_0', 'resting ecg_1',
       'resting ecg_2', 'exercise angina_0', 'exercise angina_1', 'ST slope_0',
       'ST slope_1', 'ST slope_2', 'ST slope_3', 'target_0', 'target_1'],
      dtype='object')


In [5]:
# Identificar todas as colunas-alvo
colunas_target = [col for col in df_encoded.columns if col.startswith("target_")]

# Identificar todas as colunas One-Hot
colunas_one_hot = [col for col in df_encoded.columns if col not in colunas_target]

# Iterar sobre cada target
resultados = {}

for target in colunas_target:
    print(f"\n=== Avaliando Target: {target} ===")

    for coluna in colunas_one_hot:
        print(f"\nUsando a coluna {coluna} como feature para prever {target}")

        # Definir X e y
        X = df_encoded[[coluna]]  # Usar apenas a coluna atual como feature
        y = df_encoded[target]    # Usar o target atual

        # Dividir os dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Escalar os dados
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Criar o modelo KNN
        knn = KNeighborsClassifier(n_neighbors=5)

        # Treinar o modelo
        knn.fit(X_train, y_train)

        # Fazer previsões
        y_pred = knn.predict(X_test)

        # Avaliar o modelo
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Acurácia para {target} usando {coluna}: {accuracy}")

        # Salvar os resultados
        resultados[f"{target}_{coluna}"] = {
            "acuracia": accuracy,
            "relatorio": classification_report(y_test, y_pred, output_dict=True, zero_division=1)
        }

# Exibir os resultados finais
print("\n=== Resultados Finais ===")
for key, metrics in resultados.items():
    print(f"{key}: Acurácia = {metrics['acuracia']}")


=== Avaliando Target: target_0 ===

Usando a coluna sex_0 como feature para prever target_0
Acurácia para target_0 usando sex_0: 0.6722689075630253

Usando a coluna sex_1 como feature para prever target_0
Acurácia para target_0 usando sex_1: 0.4495798319327731

Usando a coluna chest pain type_1 como feature para prever target_0
Acurácia para target_0 usando chest pain type_1: 0.5588235294117647

Usando a coluna chest pain type_2 como feature para prever target_0
Acurácia para target_0 usando chest pain type_2: 0.6554621848739496

Usando a coluna chest pain type_3 como feature para prever target_0
Acurácia para target_0 usando chest pain type_3: 0.6512605042016807

Usando a coluna chest pain type_4 como feature para prever target_0
Acurácia para target_0 usando chest pain type_4: 0.7647058823529411

Usando a coluna fasting blood sugar_0 como feature para prever target_0
Acurácia para target_0 usando fasting blood sugar_0: 0.5630252100840336

Usando a coluna fasting blood sugar_1 como f

1. sex_0: Acurácia de 67,2%
2. sex_1: Acurácia de 44,96%
3. chest pain type_1: Acurácia de 55,88%
4. chest pain type_2: Acurácia de 65,55%
5. chest pain type_3: Acurácia de 65,13%
6. chest pain type_4: Acurácia de 76,47%
7. fasting blood sugar_0: Acurácia de 56,30%
8. fasting blood sugar_1: Acurácia de 55,04%
9. resting ecg_0: Acurácia de 55,04%
10. resting ecg_1: Acurácia de 47,90%
11. resting ecg_2: Acurácia de 55,04%
12. exercise angina_0: Acurácia de 74,37%
13. exercise angina_1: Acurácia de 55,04%
14. ST slope_0: Acurácia de 55,04%
15. ST slope_1: Acurácia de 78,15%
16. ST slope_2: Acurácia de 76,47%
17. ST slope_3: Acurácia de 55,04%
## Observações:
1. Melhor desempenho: A coluna ST slope_1 obteve a maior acurácia (78,15%)
2. Acurácias baixas: Algumas colunas, como sex_1, resting ecg_1 e outras, apresentam acurácias significativamente menores, indicando que essas colunas sozinhas não fornecem   informações suficientes para uma boa previsão.
3. Outras colunas: A coluna exercise angina_0 também teve uma boa performance (74,37%), enquanto chest pain type_4 teve 76,47%.
Esses resultados indicam que algumas características, como ST slope_1 e exercise angina_0, são mais úteis para prever o target_0, enquanto outras não são tão informativas.